# Load the dataset

In [27]:
import pandas as pd

In [29]:
from datasets import load_dataset
dataset_1 = load_dataset("sentence-transformers/all-nli","triplet", split = "train", cache_dir='cache_dir_dataset').to_pandas()
dataset_1.head()

,anchor,positive,negative
0,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.","A person is at a diner, ordering an omelette."
1,Children smiling and waving at camera,There are children present,The kids are frowning
2,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,The boy skates down the sidewalk.
3,Two blond women are hugging one another.,There are women showing affection.,The women are sleeping.
4,"A few people in a restaurant setting, one of t...",The diners are at a restaurant.,The people are sitting at desks in school.


In [30]:
# droping any row with null value
df = dataset_1.dropna(axis=0)
df = a = df.reset_index(drop =True) # index is all messed up after drop so we resetindex.
print("The number of Dropped rows:",dataset_1.shape[0]-df.shape[0])
null_count_dataset_1 = dataset_1.isnull().sum().sum()
null_count_df = df.isnull().sum().sum()
print("total number of null values in dataset_1 and df:",null_count_dataset_1, "and", null_count_df)

The number of Dropped rows: 0
total number of null values in dataset_1 and df: 0 and 0


In [31]:
# counting total character in dataset
total_character = 0
for index_1, row in df.iterrows():
    total_character = len(row["anchor"]) + len(row["positive"]) + len(row["negative"]) + total_character
print(total_character)

92119414


In [32]:
# counting the number of value we won't going to translate
no_translate = 0
for index_1, row in df.iterrows():
    if (len(row["anchor"]) == 0):
        no_translate += 1
    if (len(row["positive"]) == 0):
        no_translate += 1
    if (len(row["negative"]) == 0):
        no_translate += 1
print(no_translate)

0


In [33]:
# len of dataset
len(df), len(dataset_1)

(557850, 557850)

In [34]:
# rechecking the null value
null_count_df = df.isnull().sum().sum()
null_count_df

0

In [35]:
# rechecking the null value
null_count_df = df.isnull().sum().sum()
null_count_df

0

In [36]:
# Checking the max_length in whole dataset
max_length = 0
max_index = 0
for index, row in df.iterrows():
    if (len(row["anchor"])> max_length):
        max_length = len(row["anchor"])
        max_index = index
    if (len(row["positive"])> max_length):
        max_length = len(row["positive"])
        max_index = index
    if (len(row["negative"])> max_length):
        max_length = len(row["negative"])
        max_index = index
    
    
print(max_length)

1815


In [43]:
df_1 = pd.DataFrame(columns=['anchor', 'positive', 'negative'])

In [45]:
#filter for small length
limit_length = 500
for index, row in df.iterrows():
    if (len(row["anchor"])> limit_length):
        continue
    if (len(row["positive"])> max_length):
        continue
    if (len(row["negative"])> max_length):
        continue
    new_dic = {"anchor": [row["anchor"]], "positive": [row["positive"]], "negative": [row["negative"]] }
    temp_df = pd.DataFrame(new_dic)
    df_1 = pd.concat([df_1, temp_df], ignore_index = True)
    df_1.reset_index()
    if(index%10000==0):
        print(index)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


KeyboardInterrupt: 

In [49]:
len(df_1)

123342

In [ ]:
df.iloc[534860]["anchor"]

# for Initial (save the out.csv)

In [ ]:
df_2 = pd.DataFrame(columns=['anchor', 'positive', 'negative'])

In [ ]:
df_2.head()

### api call function

In [ ]:
import requests

template="""Below is instruction that describes a task, paired with input which provide further context. Write a response  that appropriately complete the request.

### Instruction:
Translate this sentences or paragraphs to Nepali: {}

### Input:


### Response:
"""

API_URL = "https://v4uczsyh1spjupue.us-east-1.aws.endpoints.huggingface.cloud"
headers = {
 "Accept" : "application/json",
 "Authorization": "Bearer hf_xxxxxxxxxx",
 "Content-Type": "application/json"
}

def api_huggingface_llm(question):
    if (len(question) > 0):
        payload = {
                "inputs": template.format(question),
                "parameters": {
                'max_new_tokens':2048,
                'temperature': 0.1,
                # 'repetition_penalty': 1,
                "return_full_text": False
            }}
        response = requests.post(API_URL, headers=headers, json=payload)
        generated_answer = response.json()
        return generated_answer[0]["generated_text"]
    else:
        return ""

In [ ]:
api_huggingface_llm("""""")

In [ ]:
%%time
for index, row in df[0:10].iterrows():
    
    nepali_anchor = api_huggingface_llm(row["anchor"])
    nepali_positive = api_huggingface_llm(row["positive"])
    nepali_negative = api_huggingface_llm(row["negative"])
    
    new_dic = {"anchor": [nepali_anchor], "positive": [nepali_positive], "negative": [nepali_negative] }
    temp_df = pd.DataFrame(new_dic)
    # display(temp_df)

    df_2 = pd.concat([df_2, temp_df], ignore_index = True)
    df_2.reset_index()
    df_2.to_csv('out.csv')
    print(index)

In [ ]:
df_2.iloc[1]["negative"]